In [ ]:
import numpy as np
import pandas as pd
import ast

In [ ]:
movies=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv')
credits=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credits.csv')

In [ ]:
credits.head()

In [ ]:
movies.head()

In [ ]:
newfile=movies.merge(credits,on='title')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#genre
#id
#keywords
#original language
# original_title
# popularity
# production_comapny
# production_countries
# release-date(not sure)

In [ ]:
newfile = newfile[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
newfile.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
newfile.dropna(inplace=True)

In [ ]:
newfile.duplicated().sum()

0

In [ ]:
def convert(obj):
  l=[]
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [ ]:
newfile['genres']=newfile['genres'].apply(convert)

In [ ]:
newfile.head()

In [ ]:
newfile['keywords']=newfile['keywords'].apply(convert)

In [ ]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [ ]:
newfile['cast'] = newfile['cast'].apply(convert3)
newfile.head()

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [ ]:
newfile['crew']=newfile['crew'].apply(fetch_director)

In [ ]:
newfile['overview'] = newfile['overview'].apply(lambda x:x.split())

In [ ]:
newfile['genres']=newfile['genres'].apply(lambda x:[i.replace(" ","") for i in x])
newfile['keywords']=newfile['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
newfile['cast']=newfile['cast'].apply(lambda x:[i.replace(" ","") for i in x])
newfile['crew']=newfile['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
newfile['tags']=newfile['overview']+newfile['genres']+newfile['keywords']+newfile['cast']+newfile['crew']

In [ ]:
newdf=newfile[['movie_id','title','tags']]

In [ ]:
newdf['tags']=newdf['tags'].apply(lambda x:" ".join(x))

In [ ]:
newdf['tags'] = newdf['tags'].apply(lambda x:x.lower())

In [ ]:
newdf.head()

In [ ]:
import nltk 

In [ ]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [ ]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
newdf['tags']=newdf['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors=cv.fit_transform(newdf['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity =cosine_similarity(vectors)

In [ ]:
def recommend(movie):
  movie_index= newdf[newdf['title']==movie].index[0]
  distances=similarity[movie_index]
  movies_list=sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
  
  for i in movies_list:
    print (newdf.iloc[i[0]].title)


In [ ]:
recommend('Quantum of Solace')